In [1]:
%load_ext autoreload
%autoreload 2

import os

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter
from generator.preprocess import *
from generator.preprocess import remove_outlier_trajectories
from generator.road_network import RoadNetwork
import glob
from tqdm import tqdm

In [4]:
# df = pd.read_csv("../datasets/trajectories/Hanover/hannover_inrix.csv")
network = RoadNetwork()
network.load("../osm_data/hanover")

In [29]:
l, c = 0, 0
for _, g in tqdm(df.groupby("id")):
    if g.shape[0] < 5:
        continue
    l += g.shape[0]
    c += 1

print(l/c)


100%|██████████| 77188/77188 [00:02<00:00, 37544.01it/s]

241.32424353095539


In [19]:
rows = []
for _, g in tqdm(df.groupby("id")):
    if g.shape[0] < 5:
        continue
    traj = g[["longitude", "latitude", "time"]].copy()
    traj.loc[:, "time"] = pd.to_datetime(traj["time"], infer_datetime_format=True).dt.floor('s')
    traj.loc[:, "time"] = traj["time"].values.astype(np.int64) // 10 ** 9
    data = traj.to_numpy()
    seq = LineString(data[:, :2])
    rows.append((seq, data[:, 2]))

processed_df = pd.DataFrame(rows, columns=["POLYLINE", "timestamp"])
processed_df

100%|██████████| 77188/77188 [03:02<00:00, 422.73it/s]


,POLYLINE,timestamp
0,"LINESTRING (9.8288 52.3326, 9.819705 52.331356...","[1565974403.0, 1565974483.0, 1565974827.0, 156..."
1,"LINESTRING (9.6472 52.2823, 9.643848 52.283978...","[1567075203.0, 1567075233.0, 1567075263.0, 156..."
2,"LINESTRING (9.7376 52.5465, 9.72797 52.541855,...","[1566184436.0, 1566184574.0, 1566184736.0, 156..."
3,"LINESTRING (9.7119 52.343, 9.716048 52.344969,...","[1568621637.0, 1568621727.0, 1568621817.0, 156..."
4,"LINESTRING (9.7585 52.3363, 9.762929 52.34235,...","[1566671810.0, 1566671889.0, 1566671951.0, 156..."
...,...,...
73924,"LINESTRING (10.0369 53.3593, 10.037012 53.3582...","[1569563606.0, 1569563611.0, 1569563614.0, 156..."
73925,"LINESTRING (9.7278 52.3654, 9.727219 52.365731...","[1575029942.0, 1575029947.0, 1575029950.0, 157..."
73926,"LINESTRING (9.8726 52.9458, 9.718267 52.63466,...","[1573224957.0, 1573225556.0, 1573225660.0, 157..."
73927,"LINESTRING (9.738963 52.382663, 9.738996 52.38...","[1574319815.0, 1574319820.0, 1574319825.0, 157..."


In [22]:
city_bounds = network.bounds
df_clipped = clip_trajectories(processed_df.copy(), city_bounds, polyline_convert=True)
df_clipped = filter_min_points(df_clipped, 5)
df_clipped

Dask Apply:   0%|          | 0/32 [00:00<?, ?it/s]

,timestamp,POLYLINE,coords
0,"[1565974403.0, 1565974483.0, 1565974827.0, 156...","LINESTRING (9.82880 52.33260, 9.81971 52.33136...","[(9.8288, 52.3326), (9.819705, 52.331356), (9...."
1,"[1567075203.0, 1567075233.0, 1567075263.0, 156...","LINESTRING (9.64842 52.30311, 9.65059 52.30557...","[(9.648424983656717, 52.3031091), (9.650593, 5..."
3,"[1568621637.0, 1568621727.0, 1568621817.0, 156...","LINESTRING (9.71190 52.34300, 9.71605 52.34497...","[(9.7119, 52.343), (9.716048, 52.344969), (9.7..."
4,"[1566671810.0, 1566671889.0, 1566671951.0, 156...","LINESTRING (9.75850 52.33630, 9.76293 52.34235...","[(9.7585, 52.3363), (9.762929, 52.34235), (9.7..."
5,"[1574157039.0, 1574157040.0, 1574157048.0, 157...","LINESTRING (9.59739 52.43621, 9.60301 52.43416...","[(9.5973878, 52.436208161243144), (9.603011, 5..."
...,...,...,...
333628,"[1569563606.0, 1569563611.0, 1569563614.0, 156...","LINESTRING (9.72242 52.45485, 9.72219 52.45389...","[(9.722418920338985, 52.454852), (9.722186, 52..."
333629,"[1575029942.0, 1575029947.0, 1575029950.0, 157...","LINESTRING (9.72780 52.36540, 9.72722 52.36573...","[(9.7278, 52.3654), (9.727219, 52.365731), (9...."
333630,"[1573224957.0, 1573225556.0, 1573225660.0, 157...","LINESTRING (9.78251 52.45485, 9.78198 52.37981...","[(9.782512648860859, 52.454852), (9.781983, 52..."
333631,"[1574319815.0, 1574319820.0, 1574319825.0, 157...","LINESTRING (9.73896 52.38266, 9.73900 52.38260...","[(9.738963, 52.382663), (9.738996, 52.382599),..."


In [23]:
df_clipped["id"] = np.arange(1, df_clipped.shape[0]+1)
df_clipped.drop("coords", inplace=True, axis=1)
df_clipped["timestamp"] = df_clipped["timestamp"].astype(str)
df_clipped["timestamp"] = df_clipped["timestamp"].str.replace("[", "")
df_clipped["timestamp"] = df_clipped["timestamp"].str.replace("]", "")
# df_clipped["timestamp"] = df_clipped["timestamp"].str.replace(" ", ", ")
df_clipped.to_csv("../datasets/trajectories/Hanover/mapped_id_poly_clipped.csv", sep=";", index=False)

In [5]:
df = pd.read_csv("../datasets/trajectories/Hanover/mapped_id_poly_clipped.csv", sep=";")
df

,timestamp,POLYLINE,id
0,1.56597440e+09 1.56597448e+09 1.56597483e+09 1...,"LINESTRING (9.8288 52.3326, 9.819705 52.331356...",1
1,1.56707520e+09 1.56707523e+09 1.56707526e+09 1...,"LINESTRING (9.648424983656717 52.3031091, 9.65...",2
2,1.56862164e+09 1.56862173e+09 1.56862182e+09 1...,"LINESTRING (9.7119 52.343, 9.716048 52.344969,...",3
3,1.56667181e+09 1.56667189e+09 1.56667195e+09 1...,"LINESTRING (9.7585 52.3363, 9.762929 52.34235,...",4
4,1.57415704e+09 1.57415704e+09 1.57415705e+09 1...,"LINESTRING (9.5973878 52.436208161243144, 9.60...",5
...,...,...,...
153571,1.56956361e+09 1.56956361e+09 1.56956361e+09 ....,"LINESTRING (9.722418920338985 52.454852, 9.722...",153572
153572,1.57502994e+09 1.57502995e+09 1.57502995e+09 1...,"LINESTRING (9.7278 52.3654, 9.727219 52.365731...",153573
153573,1.57322496e+09 1.57322556e+09 1.57322566e+09 1...,"LINESTRING (9.782512648860859 52.454852, 9.781...",153574
153574,1.57431982e+09 1.57431982e+09 1.57431982e+09 1...,"LINESTRING (9.738963 52.382663, 9.738996 52.38...",153575


In [6]:
df["POLYLINE"] = df["POLYLINE"].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs="epsg:4326", geometry="POLYLINE")

Pandas Apply:   0%|          | 0/153576 [00:00<?, ?it/s]

In [7]:
network.fmm_trajectorie_mapping(
    
    network_file="../osm_data/hanover/edges.shp",
    input_file="../datasets/trajectories/Hanover/mapped_id_poly_clipped.csv",
    output_file="../datasets/trajectories/Hanover/road-segment-mapping.txt"
)